In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [2]:
# import json
import os
# json_path = os.path.join(os.getenv("HOME"), ".kaggle/kaggle.json")
# j = json.load(open(json_path, 'r'))
# api = KaggleApi(j)
# api.authenticate()
# files = api.competition_download_files("mercedes-benz-greener-manufacturing")

In [3]:
for file_name in os.listdir(os.getcwd()):
    if file_name.endswith(".zip"):
        print(file_name)

mercedes-benz-greener-manufacturing.zip


In [3]:
# os.system("mkdir -p mercedes-benz-greener-manufacturing")
dir_abs_path = os.path.abspath("mercedes-benz-greener-manufacturing")

# from shutil import move
# for file_name in os.listdir(os.getcwd()):
#     if file_name.endswith(".zip"):
#         move(file_name, dir_abs_path)

In [4]:
os.chdir(dir_abs_path)
# !unzip mercedes-benz-greener-manufacturing.zip
# !unzip train.csv.zip
# !unzip test.csv.zip
# !unzip sample_submission.csv.zip

In [5]:
train_csv, test_csv, submission_csv = "", "", ""
for file_name in os.listdir(os.getcwd()):
    if file_name.endswith(".csv"):
        if file_name.startswith("train"):
            train_csv = os.path.abspath(file_name)
        elif file_name.startswith("test"):
            test_csv = os.path.abspath(file_name)
        elif file_name.startswith("sample"):
            submission_csv = os.path.abspath(file_name)

In [6]:
import pandas as pd
from IPython.display import display
df = pd.read_csv(train_csv)
display(df.head())

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


y - value to be predicted
X0-X385 - features of a mercedes benz

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 378 entries, ID to X385
dtypes: float64(1), int64(369), object(8)
memory usage: 12.1+ MB


hence only 8 categorical features, 369 integer type features and 1 float-type feature

In [8]:
cols, categorical_features = df.columns, []
for col in cols:
    if df[col].dtype == 'object':
        categorical_features.append(col)
print(categorical_features)

['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [9]:
# unique labels for each feature, i.e. all categories of each feature
for col in categorical_features:
    print(col, "-", len(df[col].unique()))

X0 - 47
X1 - 27
X2 - 44
X3 - 7
X4 - 4
X5 - 29
X6 - 12
X8 - 25


In [10]:
pd.get_dummies(df, drop_first=True).shape

(4209, 557)

if the total number of categories, for even 1 categorical feature are ~500, then it would correspond to 500-dimensional one-hot encoding, which will lead to increase in the dimensionality of our net feature vector, which may impact the accuracy 

A similar problem is what we would face if we decide to include all of these 557 one-hot encoded features.
According to the niculescu's solution that won the KDD2009 cup, the one-hot encoding was limited to only the 10 most frequent labels(of each categorical feature). Hence the other labels will be then clubbed into a single category, which then will be dropped.

hence the newly created 10 dummy variables would indicate whether the sample belongs to a frequently occuring label(exactly 1 label will be set to 1 for that categorical feature), or doesn't belong to any of the 10 most frequently occuring labels(all set to 0)

In [11]:
for col in categorical_features:
    print(dict(df[col].value_counts()[:10]))

{'z': 360, 'ak': 349, 'y': 324, 'ay': 313, 't': 306, 'x': 300, 'o': 269, 'f': 227, 'n': 195, 'w': 182}
{'aa': 833, 's': 598, 'b': 592, 'l': 590, 'v': 408, 'r': 251, 'i': 203, 'a': 143, 'c': 121, 'o': 82}
{'as': 1659, 'ae': 496, 'ai': 415, 'm': 367, 'ak': 265, 'r': 153, 'n': 137, 's': 94, 'f': 87, 'e': 81}
{'c': 1942, 'f': 1076, 'a': 440, 'd': 290, 'g': 241, 'e': 163, 'b': 57}
{'d': 4205, 'a': 2, 'b': 1, 'c': 1}
{'v': 231, 'w': 231, 'q': 220, 'r': 215, 's': 214, 'd': 214, 'n': 212, 'm': 208, 'p': 208, 'i': 207}
{'g': 1042, 'j': 1039, 'd': 625, 'i': 488, 'l': 478, 'a': 206, 'h': 190, 'k': 43, 'c': 38, 'b': 28}
{'j': 277, 's': 255, 'f': 243, 'n': 242, 'i': 237, 'e': 225, 'r': 219, 'a': 210, 'w': 196, 'v': 194}


In [12]:
import numpy as np
df_copy = df.copy()
for col in categorical_features:
    top_10 = list(df[col].value_counts()[:10].keys())
    for label in top_10:
        df_copy[col+"_"+label] = np.where(df[col] == label, 1, 0)
df_copy.drop(categorical_features, axis=1, inplace=True)
for col in categorical_features:
    top_10 = list(df[col].value_counts()[:10].keys())
    top_10 = list(map(lambda x: col+"_"+x, top_10))
    display(df_copy.loc[:, top_10].head(10))

,X0_z,X0_ak,X0_y,X0_ay,X0_t,X0_x,X0_o,X0_f,X0_n,X0_w
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0


,X1_aa,X1_s,X1_b,X1_l,X1_v,X1_r,X1_i,X1_a,X1_c,X1_o
0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0
7,0,0,0,1,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,0


,X2_as,X2_ae,X2_ai,X2_m,X2_ak,X2_r,X2_n,X2_s,X2_f,X2_e
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,1
7,1,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0


,X3_c,X3_f,X3_a,X3_d,X3_g,X3_e,X3_b
0,0,0,1,0,0,0,0
1,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0
5,1,0,0,0,0,0,0
6,0,1,0,0,0,0,0
7,0,1,0,0,0,0,0
8,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0


,X4_d,X4_a,X4_b,X4_c
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
5,1,0,0,0
6,1,0,0,0
7,1,0,0,0
8,1,0,0,0
9,1,0,0,0


,X5_v,X5_w,X5_q,X5_r,X5_s,X5_d,X5_n,X5_m,X5_p,X5_i
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0


,X6_g,X6_j,X6_d,X6_i,X6_l,X6_a,X6_h,X6_k,X6_c,X6_b
0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,1,0,0,0
7,0,1,0,0,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,0


,X8_j,X8_s,X8_f,X8_n,X8_i,X8_e,X8_r,X8_a,X8_w,X8_v
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0
6,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,0


<font size="5" color="green">Advantages of One-hot encoding ONLY the top-10 labels per category</font>
* straightforward to implement
* doesn't require in-depth variable exploration
* prevents massive expansion of the feature space

<font size="5" color="red">DISAdvantages of One-hot encoding ONLY the top-10 labels per category</font>
* doesn't add any information that may make the variable productive
* ignores the information from the less frequent labels

hence the other labels per category which appear less frequently, are regarded as NOISE.

# Types of Encoding

1. Nominal Encoding - corresponds to nominal features
2. Ordinal Encoding - corresponds to ordinal features

## Nominal Features
* no need to worry about arrangement of the categories in a feature

## Ordinal Features
* arrangement of the categories in a feature is important
* each category of an ordinal feature has some rank associated to it
* suppose salary prediction based on a feature= education
 * education = BE, Bcom, Masters, PHD
 * assumption = salary-order : PHD > Masters > BE > Bcom
 * hence PHD - 1, Masters - 2, BE - 3, Bcom - 4
 
# Nominal Encoding
3 basic techniques:
1. One-hot encoding
2. One-hot encoding with many categorical variables
3. Mean encoding

# One-hot encoding
1. one-hot vector representations
2. number of columns = number of unique categories for a feature
3. column set to 1 if the sample's feature value is that particular label, else set to 0
4. the new columns created as a result are called as *dummy variables*

## Dummy variable trap
1. suppose we have 3 categories for a feature, A, B, C.
2. instead of having 3 columns, we can have 2 columns, A, B, where 
    1. if the label = A, then the column A can be set to 1 and B will be set to 0
    2. if label = B, then the column A can be set to 0 and B will be set to 1
    3. if label = C, then both columns A and B will be set to 1
3. hence, **we use the drop_first=True**, while using the *pandas.get_dummies* function

## <font color="red">DISadvantages</font>
1. the obvious increase in dimensionality , which causes <font color="red">curse of dimensionality</font>

# One-hot encoding *with multiple categories*
1. since we had decided that to avoid the *curse of dimensionality*, avoid one-hot encoding for features with numerous categories, we need to have some other way to encode them
2. one way to go about such an issue is to use the *top-10 categories encode only* method, which was done during the KDD-orange competition 

# Mean encoding
1. calculate the mean value of output(also called target variable) per category for such a feature
2. assign the mean value as a value that represents this feature
3. this is also used when the number of categories is just too large for one-hot encoding to be properly used
4. in addition to this, a new column that denotes the standard deviation(or variance) of the target variable w.r.t. the same category can also be introduced, to make up for low-predicitve power.

# Ordinal Encoding
2 basic techniques:
1. Label Encoding
2. Target-guided ordinal encoding

[Follow this for more](https://www.kaggle.com/scirpus/ordinal-encoding/comments)

# Label Encoding
1. highest importance/lowest rank(lowest integral value, i.e. rank = 1 means it should be assigned the highest available value)
    1. for the example of education = BE, Bcom, Masters, PHD, the corresponding values will be PHD=4, Masters=3, BE=2, Bcom=1
2. use [sklearn.preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) for this

# Target-guided ordinal encoding
1. try to find out the mean of the output(to be predicted) values for each category of such a feature
    1. this should be done only after converting the output variables into an integral form, if they weren't in the first place
2. if suppose c = number of categories a feature has, then there will be C means to compare from
3. based on these C number of means, assign top rank(superior, large number, just like PHD in the label-encoding example) to a category if it has the highest mean of all the remaining unassigned categories.

<font size="6" >Why do we require feature scaling?</font>
1. if suppose feature values are always large, for non-categorical features, then it may seem for some model(s) that the samples are too distant/too different
    1. for instance, consider the kNN classifier, if suppose height = cm, weight = kg, predicted-label = BMI
    2. consider 2 points, h1 = 180cm, w1 = 78kg, h2 = 170cm, w2 = 84kg
    3. for kNN, it will plot the points as x vs. y plots, and it would see that the euclidean distances between these points is large
    4. hence there is a need to scale down these values
2. there are many different types of scaling, the most common being normalisation across each feature

<font size="4.5">Models that would require feature scaling:</font>
* Linear Regression
    * convergence can take place quickly, since before scaling, it may seem that the model's current state is far from the minima, but after scaling, it may seem that its actually much closer(this is all because of small values vs larger values being compared)
* Models that use euclidean distances, like kNN classifier and K-Means clustering
* Deep-learning models

<font size="4.5">Models that would </font><font color="red" size="4.5">NOT</font><font size="4.5"> require feature scaling:</font>
* Decision Trees - number of branches will remain the same, scaling down wont change the tree structure
* RandomForest
* XGBoost

# Handle missing values in categorical variables
1. Delete the rows
    1. delete the row that has a null value for a particular categorical variable
    2. obvious consequence is reduction of the train size, and also if the test data contains null values for this particular variable, then the model doesn't know of how to handle prediction of an input sample with this variable set to NULL, thus messing up the accuracy
    3. it may also happen that the rows being deleted by one categorical feature, may impact special knowledge which would have been otherwise derived by the model, had the rows been not deleted, w.r.t. some other feature.

2. replace with the most frequent values, i.e. the mode for that categorical variable
    1. already saw in the house-prices advanced regression technique problem that usage of mode of the train data, to fill the null values for both test and train data yielded better score than the usage of mode of train data to fill the null values for train data and usage of mode of test-data to fill its own null values
    2. this may lead to an **imbalanced dataset**, since a particular value may be repeating too many times, but the presence of the other value might be a key aspect of learning/prediction.
3. use a classifier
    1. to predict what the null value should be, from the other available features for this sample, use a classifier
    2. we can include the output variable as an independent variable as well, in this particular classification
4. apply an unsupervied-ML technique
    1. if clustering is to be used, then use all other feature vectors for clustering, to create N clusters, where each cluster represents a particular category of this missing-value feature.

# Frequency Encoding
* replace each label of a categorical variable by a count - amount of times that label has appeared in the dataset
* usually to be used when a categorical feature has to many categories, due to which one-hot encoding is certain to cause *curse of dimensionality*
* frequency(count/total_count) can also be used, instead of the actual count value
* valid coding in case of GBM based models like xgboost. <font color="Red">How?</font>
* **KDD-2014 solutions**

In [13]:
df_freq = df.copy()

In [17]:
for col in categorical_features:
    d = dict(df_freq[col].value_counts())
    df_freq[col] = df_freq[col].map(d)
display(df_freq.head(10))

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,11,408,6,440,4205,1,1039,163,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,11,31,4,163,4205,1,478,163,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,175,52,137,1942,4205,2,1039,105,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,175,31,137,1076,4205,2,478,225,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,175,408,137,1076,4205,1,625,242,...,0,0,0,0,0,0,0,0,0,0
5,18,92.93,306,592,81,1942,4205,1,190,255,...,0,0,1,0,0,0,0,0,0,0
6,24,128.76,67,251,81,1076,4205,7,190,255,...,0,0,0,0,0,0,0,0,0,0
7,25,91.91,269,590,1659,1076,4205,7,1039,210,...,0,0,0,0,0,0,0,0,0,0
8,27,108.67,182,598,1659,163,4205,7,488,117,...,1,0,0,0,0,0,0,0,0,0
9,30,126.99,181,592,63,1942,4205,7,206,225,...,0,0,1,0,0,0,0,0,0,0


<font size="5" color="green">Advantages of frequency encoding</font>
1. simple to implement
2. does not increase the feature-dimensional space

<font size="5" color="red">DISAdvantages of frequency encoding</font>
1. if labels do have the same frequency value, then such features will loose uniqueness information with respect to a sample belonging to label-1 or label-2.
2. the numbers thus added may not be related to the predictive power.

Alternatives would be to use MCA(PCA for categorical features)

# Hashing Trick
1. [follow this link for now](https://booking.ai/dont-be-tricked-by-the-hashing-trick-192a6aae3087)